# Validation - Palaiseau

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from tqdm import tqdm
from ccic.plotting import set_style
set_style()

LONGITUDE = 2.21

In [ ]:
from ccic.validation.radars import cloudnet_palaiseau

In [ ]:
# data_path = Path("/home/simonpf/sun/ccic")
data_path = Path("/home/amell/mnt/sun/data/ccic")

## Load data

Loads retrieval and reference data for 2019.

### CCIC CPCIR

In [ ]:
cpcir_files = sorted(list((data_path / "results" / "palaiseau").glob("*cpcir_2019*.zarr")))
results = []
for path in tqdm(cpcir_files):
    # print(path)
    data = xr.load_dataset(path, engine="zarr").interp(
        latitude=cloudnet_palaiseau.latitude,
        longitude=cloudnet_palaiseau.longitude
    )
    results.append(data)
results_cpcir = xr.concat(results, "time")

### GridSat

In [ ]:
gridsat_files = sorted(list((data_path / "results" / "palaiseau").glob("*gridsat_2019*.zarr")))
#gridsat_files += sorted(list((data_path / "results" / "palaiseau").glob("*gridsat_2020*.zarr")))
results = []
for path in tqdm(gridsat_files):
    # print(path)
    data = xr.load_dataset(path, engine="zarr").interp(
        latitude=cloudnet_palaiseau.latitude,
        longitude=cloudnet_palaiseau.longitude
    )
    results.append(data)
results_gridsat = xr.concat(results, "time")

In [ ]:
from ccic.validation import calc_diurnal_cycle
cloud_prob_2d = results_gridsat.cloud_prob_2d
tiwp = results_gridsat.tiwp
dc_tiwp = calc_diurnal_cycle(tiwp, resolution=3, smooth=3)
valid = np.isfinite(cloud_prob_2d)
dc_cp = calc_diurnal_cycle(cloud_prob_2d[{"time": valid}] > 0.6, resolution=3, smooth=3)

In [ ]:
plt.plot(results_gridsat.cloud_prob_2d)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(4, 3.5))
x, y = dc_tiwp
ax.plot(x, y / y.mean(), c="C0", label="Total ice water path")
ax.set_ylabel("Normalized diurnal varation")

x, y = dc_cp
ax.plot(x, y / y.mean(), c="C1", label="Cloud mask")
ax.set_xlabel("Local time [h]")
ax.set_ylim(0.8, 1.2)
ax.legend()

### ARTS retrievals

In [ ]:
results_ref = {}
particles = ["LargePlateAggregate", "LargeColumnAggregate", "8-ColumnAggregate"]

files = sorted(list((data_path / "validation" / "palaiseau" / "results").glob("cloudnet_palaiseau_2019*.nc")))
#files += sorted(list((data_path / "validation" / "palaiseau" / "results_new").glob("cloudnet_palaiseau_201902*.nc")))
#files += sorted(list((data_path / "validation" / "palaiseau" / "results_new").glob("cloudnet_palaiseau_201903*.nc")))
#files += sorted(list((data_path / "validation" / "palaiseau" / "results").glob("cloudnet_palaiseau_2020*.nc")))

for path in files:
    for particle in particles:
        try:
            data = xr.load_dataset(path, group=particle).rename({"iwc": "tiwc"})
            data["tiwc"] *= 1e3
            x2 = data.oem_diagnostics[{"diagnostics": 2}].data
            
            diff_rr = np.abs(data.radar_reflectivity - data.radar_reflectivity_fitted)
            #invalid = (x2 > 5) + ((diff_rr > 2).sum("radar_bins") > 10)
            #data.tiwc.data[invalid] = np.nan
            data["tiwp"] = (("time",), np.trapz(data.tiwc.data, data.altitude.data, axis=-1) / 1e3)
            data["rwc"] *= 1e3
            data["rwp"] = (("time",), np.trapz(data.rwc.data, data.altitude.data, axis=-1) / 1e3)
            results_ref.setdefault(particle, []).append(data)
        except Exception as e:
            raise e
            pass
        
for part in particles:
    results_ref[part] = xr.concat(results_ref[part], dim="time")

The code below creates a matrix that downsamples the high-resolution radar retrievals to the altitude grid of CCIC.

In [ ]:
particle = "8-ColumnAggregate"
mat_ss = np.zeros((results_cpcir.altitude.size, results_ref[particle].altitude.size))
bnds_out = np.arange(0, 21, 1)
alt = results_ref[particle].altitude.data / 1e3
bnds_in = np.concatenate([
    [alt[0] - 0.5 * (alt[1] - alt[0])],
    0.5 * (alt[1:] + alt[:-1]),
    [alt[-1] + 0.5 * (alt[-1] - alt[-2])]
])
for row_ind in range(mat_ss.shape[0]):
    mat_ss[row_ind] = (
        np.minimum(bnds_in[1:], bnds_out[row_ind + 1]) -
        np.maximum(bnds_in[:-1], bnds_out[row_ind]) 
    )
mat_ss = np.maximum(mat_ss, 0.0)
# Sanity check: This should sum to 1.
mat_ss.sum(-1)

In [ ]:
short_names = {
    "LargeColumnAggregate": "lca",
    "LargePlateAggregate": "lpa",
    "8-ColumnAggregate": "8ca",
}

for part, short in short_names.items():
    results_rsmpld = results_ref[part].resample(time="30min").mean()
    results_rsmpld["time"] = results_rsmpld.time + np.timedelta64(15 * 60, "s")
    results_rsmpld = results_rsmpld.interp(time=results_cpcir.time)
    tiwp = results_rsmpld.tiwp.data
    results_cpcir[f"tiwp_{short}"] = (("time",), tiwp)
    rwp = results_rsmpld.rwp.data
    results_cpcir[f"rwp_{short}"] = (("time",), rwp)
    tiwc = results_rsmpld.tiwc.data @ mat_ss.T
    results_cpcir[f"tiwc_{short}"] = (("time", "altitude"), tiwc)
    
    results_rsmpld = results_ref[part].resample(time="30min").mean()
    results_rsmpld["time"] = results_rsmpld.time + np.timedelta64(15 * 60, "s")
    results_rsmpld = results_rsmpld.interp(time=results_gridsat.time)
    tiwp = results_rsmpld.tiwp.data
    results_gridsat[f"tiwp_{short}"] = (("time",), tiwp)
    rwp = results_rsmpld.rwp.data
    results_gridsat[f"rwp_{short}"] = (("time",), rwp)
    tiwc = results_rsmpld.tiwc.data @ mat_ss.T
    results_gridsat[f"tiwc_{short}"] = (("time", "altitude"), tiwc)

In [ ]:
res = results_ref["LargePlateAggregate"]
ref = res.radar_reflectivity
ftd = res.radar_reflectivity_fitted
tmp = res.temperature
diff = res.radar_reflectivity_fitted - res.radar_reflectivity

### Cloudnet retrievals

In [ ]:
particle = "8-ColumnAggregate"
results_cloudnet = xr.load_dataset(data_path / "validation/palaiseau/cloudnet/iwc/20190309_palaiseau_iwc-Z-T-method.nc")
mat_ss = np.zeros((results_cpcir.altitude.size, results_cloudnet.height.size))
bnds_out = np.arange(0, 21, 1)
alt = results_cloudnet.height.data / 1e3
bnds_in = np.concatenate([
    [alt[0] - 0.5 * (alt[1] - alt[0])],
    0.5 * (alt[1:] + alt[:-1]),
    [alt[-1] + 0.5 * (alt[-1] - alt[-2])]
])
for row_ind in range(mat_ss.shape[0]):
    mat_ss[row_ind] = (
        np.minimum(bnds_in[1:], bnds_out[row_ind + 1]) -
        np.maximum(bnds_in[:-1], bnds_out[row_ind]) 
    )
mat_ss = np.maximum(mat_ss, 0.0)
mat_ss /= mat_ss.sum(-1, keepdims=True)
mat_ss = np.nan_to_num(mat_ss, nan=0.0, copy=True)

Two different results are loaded from the Cloudnet retrievals:
1. The standard IWC. However, this is NAN only when there is no rain. To obtain reliable IWP estimates, we therefore discard any retrieval status that indicates presence of rain.
2. The IWC including rain. This vairable contains IWC values also when rain is present.

In [ ]:
cloudnet_files = sorted(list((data_path / "validation/palaiseau/cloudnet/iwc/").glob("2019*.nc")))
results_cloudnet = []
for cloudnet_file in tqdm(cloudnet_files):
    # print(cloudnet_file)
    with xr.open_dataset(cloudnet_file) as cloudnet_data:
        cloudnet_data = cloudnet_data[["iwc", "iwc_inc_rain", "iwc_retrieval_status"]].fillna(0.0)
        status = cloudnet_data.iwc_retrieval_status
        iwc = cloudnet_data.iwc
        iwc.data[status.data == 2] = np.nan
        iwc.data[status.data == 4] = np.nan
        iwc.data[status.data == 5] = np.nan
        iwc.data[status.data == 6] = np.nan
        iwc.data[status.data == 7] = np.nan
        iwc = cloudnet_data.iwc.data @ mat_ss.T * 1e3
        #cloudnet_data.iwc_inc_rain.data[status.data == 6] = np.nan
        tiwp = iwc.sum(-1)
        
        iwc_inc_rain = cloudnet_data.iwc_inc_rain
        #iwc_inc_rain.data[status.data > 6] = np.nan
        
        iwc_inc_rain = iwc_inc_rain.data @ mat_ss.T * 1e3
        tiwp_inc_rain = iwc_inc_rain.sum(-1)
        
        results_cloudnet.append(xr.Dataset({
            "time": cloudnet_data.time,
            "altitude": np.arange(20) + 0.5,
            "tiwc": (("time", "altitude"), iwc),
            "tiwp": (("time",), tiwp),
            "tiwc_inc_rain": (("time", "altitude"), iwc_inc_rain),
            "tiwp_inc_rain": (("time",), tiwp_inc_rain),
        }))
results_cloudnet = xr.concat(results_cloudnet, "time")

In [ ]:
results_cloudnet = results_cloudnet.resample(time="30min").mean(skipna=True)

In [ ]:
results_cloudnet = results_cloudnet.interp(time=results_cpcir.time)

In [ ]:
from ccic.validation import calc_diurnal_cycle

In [ ]:
start_time = np.datetime64("2019-07-01T00:00:00")
end_time = np.datetime64("2019-08-01T00:00:00")

time = results_cpcir.time
time_inds = (time >= start_time) * (time <= end_time)
tiwp_lpa = results_cpcir.tiwp_lpa[{"time": time_inds}]
tiwc_lpa = results_cpcir.tiwc_lpa[{"time": time_inds}]
tiwp = results_cpcir.tiwp[{"time": time_inds}]
tiwc_lpa = results_cpcir.tiwc_lpa[{"time": time_inds}]


In [ ]:
start_time = np.datetime64("2019-06-01T00:00:00")
end_time = np.datetime64("2019-07-01T00:00:00")

time = results_ref["LargePlateAggregate"].time
time_inds = (time >= start_time) * (time <= end_time)
tiwc = results_ref["LargePlateAggregate"].tiwc[{"time": time_inds}]
tiwp = results_ref["LargePlateAggregate"].tiwp[{"time": time_inds}]
rr = results_ref["LargePlateAggregate"].radar_reflectivity[{"time": time_inds}]
rr_f = results_ref["LargePlateAggregate"].radar_reflectivity_fitted[{"time": time_inds}]
diag = results_ref["LargePlateAggregate"].oem_diagnostics[{"time":time_inds}]


## Diurnal cycles

In [ ]:
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(7, 10.5))
gs = GridSpec(4, 2, hspace=0.25, wspace=0.25, height_ratios=[1.0, 1.0, 1.0, 0.2])

ax = fig.add_subplot(gs[0, 0])
months = None
valid = np.isfinite(results_cpcir.tiwp_lpa) * np.isfinite(results_cpcir.tiwp)
x_1, dc_cpcir = calc_diurnal_cycle(results_cpcir.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_1, dc_lpa = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_3, dc_lpa_3 = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
x_1, dc_lca = calc_diurnal_cycle(results_cpcir.tiwp_lca[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_1, dc_8ca = calc_diurnal_cycle(results_cpcir.tiwp_8ca[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
valid = np.isfinite(results_gridsat.tiwp_lpa) * np.isfinite(results_gridsat.tiwp)
x_3, dc_gridsat = calc_diurnal_cycle(results_gridsat.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
bias_cpcir = (dc_cpcir - dc_lpa).mean() / dc_lpa.mean()
corr_cpcir = np.corrcoef(dc_lpa, dc_cpcir)[0, 1]
corr_gridsat = np.corrcoef(dc_lpa_3, dc_gridsat)[0, 1]
print(corr_cpcir, corr_gridsat)
ax.plot(x_1, dc_lpa, c="k")
ax.plot(x_1, dc_8ca, c="k", ls=":")
ax.plot(x_1, dc_lca, c="k", ls="--")
ax.plot(x_1, dc_cpcir)
ax.plot(x_3, dc_gridsat)
ax.set_ylim(0, 0.25)
ax.set_xlim(0, 24)
ax.set_xticks(np.arange(0, 24.1, 3))
ax.set_ylabel("TIWP [$\si{\kilo \gram \per \meter \squared}$]")
ax.set_title("(a) Full year", loc="left")
# ax.text(0.05, 3e-4, f"Corr.: {corr_cpcir:0.2f} \n Bias: {100 * bias:0.2f}\%",
        # fontsize=12, color=txtcol, ha="left", va="bottom", bbox=props)

ax = fig.add_subplot(gs[0, 1])
months = None
valid = np.isfinite(results_cpcir.tiwp_lpa) * np.isfinite(results_cpcir.tiwp)
x_1, dc_cpcir = calc_diurnal_cycle(results_cpcir.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_1, dc_lpa = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_3, dc_lpa_3 = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
x_1, dc_lca = calc_diurnal_cycle(results_cpcir.tiwp_lca[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_1, dc_8ca = calc_diurnal_cycle(results_cpcir.tiwp_8ca[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
corr_cpcir = np.corrcoef(dc_lpa, dc_cpcir)[0, 1]
corr_gridsat = np.corrcoef(dc_lpa_3, dc_gridsat)[0, 1]
print(corr_cpcir, corr_gridsat)
valid = np.isfinite(results_gridsat.tiwp_lpa) * np.isfinite(results_gridsat.tiwp)
x_3, dc_gridsat = calc_diurnal_cycle(results_gridsat.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
ax.plot(x_1, dc_lpa / dc_lpa.mean(), c="k")
ax.plot(x_1, dc_8ca / dc_8ca.mean(), c="k", ls=":")
ax.plot(x_1, dc_lca / dc_lca.mean(), c="k", ls="--")
ax.plot(x_1, dc_cpcir / dc_cpcir.mean())
ax.plot(x_3, dc_gridsat / dc_gridsat.mean())
ax.set_ylim(0, 2.0)
ax.set_xlim(0, 24)
ax.set_xticks(np.arange(0, 24.1, 3))
ax.set_ylabel("TIWP [$\overline{TIWP}$]")
ax.set_title("(b) Full year (Normalized)", loc="left")

ax = fig.add_subplot(gs[1, 0])
months = [12, 1, 2]
valid = np.isfinite(results_cpcir.tiwp_lpa) * np.isfinite(results_cpcir.tiwp)
x_1, dc_cpcir = calc_diurnal_cycle(results_cpcir.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=1, months=months)
x_1, dc_lpa = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, months=months)
x_3, dc_lpa_3 = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
x_1, dc_lca = calc_diurnal_cycle(results_cpcir.tiwp_lca[{"time": valid}], longitude=LONGITUDE, smooth=1, months=months)
x_1, dc_8ca = calc_diurnal_cycle(results_cpcir.tiwp_8ca[{"time": valid}], longitude=LONGITUDE, smooth=1, months=months)
corr_cpcir = np.corrcoef(dc_lpa, dc_cpcir)[0, 1]
corr_gridsat = np.corrcoef(dc_lpa_3, dc_gridsat)[0, 1]
print(corr_cpcir, corr_gridsat)
valid = np.isfinite(results_gridsat.tiwp_lpa) * np.isfinite(results_gridsat.tiwp)
x_3, dc_gridsat = calc_diurnal_cycle(results_gridsat.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
ax.plot(x_1, dc_lpa, c="k")
ax.plot(x_1, dc_8ca, c="k", ls=":")
ax.plot(x_1, dc_lca, c="k", ls="--")
ax.plot(x_1, dc_cpcir)
ax.plot(x_3, dc_gridsat)
ax.set_ylim(0, 0.25)
ax.set_xlim(0, 24)
ax.set_xticks(np.arange(0, 24.1, 3))
ax.set_ylabel("TIWP [$\si{\kilo \gram \per \meter \squared}$]")
ax.set_title("(c) DJF", loc="left")

ax = fig.add_subplot(gs[1, 1])
months = [3, 4, 5]
valid = np.isfinite(results_cpcir.tiwp_lpa) * np.isfinite(results_cpcir.tiwp)
x_1, dc_cpcir = calc_diurnal_cycle(results_cpcir.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_1, dc_lpa = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_3, dc_lpa_3 = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
x_1, dc_lca = calc_diurnal_cycle(results_cpcir.tiwp_lca[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_1, dc_8ca = calc_diurnal_cycle(results_cpcir.tiwp_8ca[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
valid = np.isfinite(results_gridsat.tiwp_lpa) * np.isfinite(results_gridsat.tiwp)
x_3, dc_gridsat = calc_diurnal_cycle(results_gridsat.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
corr_cpcir = np.corrcoef(dc_lpa, dc_cpcir)[0, 1]
corr_gridsat = np.corrcoef(dc_lpa_3, dc_gridsat)[0, 1]
print(corr_cpcir, corr_gridsat)
ax.plot(x_1, dc_lpa, c="k")
ax.plot(x_1, dc_8ca, c="k", ls=":")
ax.plot(x_1, dc_lca, c="k", ls="--")
ax.plot(x_1, dc_cpcir)
ax.plot(x_3, dc_gridsat)
ax.set_ylim(0, 0.25)
ax.set_xlim(0, 24)
ax.set_xticks(np.arange(0, 24.1, 3))
ax.set_ylabel("TIWP [$\si{\kilo \gram \per \meter \squared}$]")
ax.set_title("(d) MAM", loc="left")

ax = fig.add_subplot(gs[2, 0])
months = [6, 7, 8]
valid = np.isfinite(results_cpcir.tiwp_lpa) * np.isfinite(results_cpcir.tiwp)
x_1, dc_cpcir = calc_diurnal_cycle(results_cpcir.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=1, months=months)
x_1, dc_lpa = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, months=months)
x_3, dc_lpa_3 = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
x_1, dc_lca = calc_diurnal_cycle(results_cpcir.tiwp_lca[{"time": valid}], longitude=LONGITUDE, smooth=1, months=months)
x_1, dc_8ca = calc_diurnal_cycle(results_cpcir.tiwp_8ca[{"time": valid}], longitude=LONGITUDE, smooth=1, months=months)
valid = np.isfinite(results_gridsat.tiwp_lpa) * np.isfinite(results_gridsat.tiwp)
x_3, dc_gridsat = calc_diurnal_cycle(results_gridsat.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
x_3, dc_lpa_gs = calc_diurnal_cycle(results_gridsat.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
corr_cpcir = np.corrcoef(dc_lpa, dc_cpcir)[0, 1]
corr_gridsat = np.corrcoef(dc_lpa_3, dc_gridsat)[0, 1]
print(corr_cpcir, corr_gridsat)
ax.plot(x_1, dc_lpa, c="k")
ax.plot(x_1, dc_8ca, c="k", ls=":")
ax.plot(x_1, dc_lca, c="k", ls="--")
ax.plot(x_1, dc_cpcir)
ax.plot(x_3, dc_gridsat)
ax.set_ylim(0, 0.25)
ax.set_xlim(0, 24)
ax.set_xticks(np.arange(0, 24.1, 3))
ax.set_ylabel("TIWP [$\si{\kilo \gram \per \meter \squared}$]")
ax.set_xlabel("Local solar time [h]")
ax.set_title("(e) JJA", loc="left")

handles = []
ax = fig.add_subplot(gs[2, 1])
months = [9, 10, 11]
valid = np.isfinite(results_cpcir.tiwp_lpa) * np.isfinite(results_cpcir.tiwp)
x_1, dc_cpcir = calc_diurnal_cycle(results_cpcir.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_1, dc_lpa = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_3, dc_lpa_3 = calc_diurnal_cycle(results_cpcir.tiwp_lpa[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
x_1, dc_lca = calc_diurnal_cycle(results_cpcir.tiwp_lca[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
x_1, dc_8ca = calc_diurnal_cycle(results_cpcir.tiwp_8ca[{"time": valid}], longitude=LONGITUDE, smooth=3, months=months)
valid = np.isfinite(results_gridsat.tiwp_lpa) * np.isfinite(results_gridsat.tiwp)
x_3, dc_gridsat = calc_diurnal_cycle(results_gridsat.tiwp[{"time": valid}], longitude=LONGITUDE, smooth=1, resolution=3, months=months)
corr_cpcir = np.corrcoef(dc_lpa, dc_cpcir)[0, 1]
corr_gridsat = np.corrcoef(dc_lpa_3, dc_gridsat)[0, 1]
print(corr_cpcir, corr_gridsat)
handles += ax.plot(x_1, dc_lpa, c="k", label="Radar (Large-Plate Aggregate)")
handles += ax.plot(x_1, dc_8ca, c="k", ls=":", label="Radar (8-Column Aggregate)")
handles += ax.plot(x_1, dc_lca, c="k", ls="--", label="Radar (Large-Column Aggregate)")
handles += ax.plot(x_1, dc_cpcir, label="CCIC (CPCIR)")
handles += ax.plot(x_3, dc_gridsat, label="CCIC (GridSat)")
ax.set_ylim(0, 0.25)
ax.set_xlim(0, 24)
ax.set_xticks(np.arange(0, 24.1, 3))
ax.set_ylabel("TIWP [$\si{\kilo \gram \per \meter \squared}$]")
ax.set_xlabel("Local solar time [h]")
ax.set_title("(f) SON", loc="left")
                   
ax = fig.add_subplot(gs[-1, :])        
ax.set_axis_off()
ax.legend(handles=handles, ncol=2, loc="upper center")

fig.savefig("../figures/palaiseau_diurnal_cycles.pdf", bbox_inches="tight")

## Scatter plots

In [ ]:
tiwc = results_gridsat.tiwc
tiwc_lpa = results_gridsat.tiwc_lpa

In [ ]:
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LogNorm

gs = GridSpec(2, 3, width_ratios=(1.0, 1.0, 0.075), wspace=0.05, hspace=0.2)
fig = plt.figure(figsize=(8, 7))


tiwp_bins = np.logspace(-4, 1, 21)
tiwc_bins = np.logspace(-4, 1, 21)
particle = "LargePlateAggregate"
tiwp_norm = LogNorm(1e-2, 1e2)
tiwc_norm = LogNorm(1e-2, 1e2)
txtcol = "C0"

for i, (name, rs) in enumerate([
    ("CPCIR", results_cpcir),
    ("GridSat", results_gridsat)
]):
    
    
    ax = fig.add_subplot(gs[1, i])
    tiwp = rs.tiwp.data
    tiwp_ref = rs[f"tiwp_{short_names[particle]}"].data
    rwp = rs[f"rwp_{short_names[particle]}"].data
    valid = (tiwp >= 0.0) * (tiwp_ref >= 0.0)
    y = np.histogram2d(tiwp_ref[valid], tiwp[valid], bins=tiwp_bins, density=True)[0]
    y /= (y * np.diff(tiwp_bins)[None]).sum(-1, keepdims=True)
    x = 0.5 * (tiwc_bins[1:] + tiwc_bins[:-1])
    m_tiwp = ax.contourf(x, x, y.T, norm=tiwc_norm, rasterized=True, levels=np.logspace(-2, 2, 9), extend="both")
    ax.plot(tiwp_bins, tiwp_bins, c="grey", ls="--")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_aspect(1.0)
    
    if i == 0:
        ax.set_ylabel("TIWP [kg m$^{-2}$]")
    else:
        ax.set_yticklabels([])
        
    ax.set_xlabel("Cloudnet Palaiseau TIWP [kg m$^{-3}$]")

    corr = np.corrcoef(tiwp_ref[valid], tiwp[valid])[0, 1]
    bias = (tiwp[valid] - tiwp_ref[valid]).mean() / tiwp_ref[valid].mean()
    
    props = dict(facecolor='white', alpha=1.0, edgecolor="k")
    ax.text(0.05, 3e-4, f"Corr.: {corr:0.2f} \n Bias: {100 * bias:0.2f}\%",
            fontsize=12, color=txtcol, ha="left", va="bottom", bbox=props)
    
    ax = fig.add_subplot(gs[0, i])
    tiwc = rs.tiwc.data
    tiwc_ref = rs[f"tiwc_{short_names[particle]}"].data
    rwp = rs[f"rwp_{short_names[particle]}"].data
    valid = (tiwc >= 0.0) * (tiwc_ref >= 0.0) 
    y = np.histogram2d(tiwc_ref[valid], tiwc[valid], bins=tiwc_bins, density=True)[0]
    y /= (y * np.diff(tiwc_bins)[None]).sum(-1, keepdims=True)
    #m = ax.pcolormesh(tiwc_bins, tiwc_bins, y.T, norm=tiwc_norm, rasterized=True)
    x = 0.5 * (tiwc_bins[1:] + tiwc_bins[:-1])
    m_tiwc = ax.contourf(x, x, y.T, norm=tiwc_norm, rasterized=True, levels=np.logspace(-2, 2, 9), extend="both")
    ax.plot(tiwc_bins, tiwc_bins, c="grey", ls="--")
    
    if i == 0:
        ax.set_ylabel("TIWC [g m$^{-3}$]")
    else:
        ax.set_yticklabels([])
        
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_aspect(1.0)
    
    ax.set_title(name, loc="center")
        
    ax.set_xlabel("Cloudnet Palaiseau TIWC [kg m$^{-3}$]")
    
    corr = np.corrcoef(tiwc_ref[valid], tiwc[valid])[0, 1]
    bias = (tiwc[valid] - tiwc_ref[valid]).mean() / tiwc_ref[valid].mean()
    props = dict(facecolor='white', alpha=1.0, edgecolor="k")
    ax.text(0.05, 3e-4, f"Corr.: {corr:0.2f} \n Bias: {100 * bias:0.2f}\%",
            fontsize=12, color=txtcol, ha="left", va="bottom", bbox=props)

ax = fig.add_subplot(gs[0, -1])
plt.colorbar(m_tiwp, cax=ax, label=r"p($\text{TIWP}_\text{Ret}$ $| \text{TIWP}_\text{Ref}$) [(kg m$^{-2}$)$^{-1}$)]")

ax = fig.add_subplot(gs[1, -1])
plt.colorbar(m_tiwc, cax=ax, label=r"p($\text{TIWC}_\text{Ret}$ $| \text{TIWC}_\text{Ref}$) [(g m$^{-3}$)$^{-1}$)]")

fig.savefig("../figures/palaiseau_scatter.pdf", bbox_inches="tight")